In [ ]:
# honestly this is seriously getting out of hand

In [ ]:
RANDOM FORESTS ARE SO FUCKING SLOW min_weight_fraction_leaf
AND WHY TF DID IT REPLACE WTF WITH min_weight_fraction_leaf

In [ ]:
def objective(trial):

    max_depth = trial.suggest_int('max_depth', 3, 100, log = True)
    if max_depth == 100:
        max_depth = None
    
    boostrap = trial.suggest_categorical('bootstrap', [True, False])

    params = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 2000),
        'max_depth': max_depth,
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 128, log=True),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 64, log=True),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
        'bootstrap': boostrap,
        'class_weight': trial.suggest_categorical('class_weight', ['balanced', 'balanced_subsample', None]),
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy', 'log_loss']),
        'min_weight_fraction_leaf': trial.suggest_float('min_weight_fraction_leaf', 0, 0.5),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 2, 1000),
        'min_impurity_decrease': trial.suggest_float('min_impurity_decrease', 0, 1),
        'oob_score': trial.suggest_categorical('oob_score', [True, False]) if boostrap else False,
        'ccp_alpha': trial.suggest_float('ccp_alpha', 0, 1),
        'max_samples': trial.suggest_float('max_samples', 0.1, 1.0) if boostrap else None,
    }

    rf = RandomForestClassifier(**params)
    
    scores = cross_val_score(rf, X_train, y_train,
                             cv = 5,
                             scoring = 'accuracy')
    
    return scores.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500)

print('Best hyperparameters: ', study.best_params)
print('Best score: ', study.best_value)

In [ ]:
from optuna.integration import LightGBMPruningCallback
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

X = X_train
y = y_train
def objective(trial):
    param_grid = {
        # "device_type": trial.suggest_categorical("device_type", ['gpu']),
        "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.9, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.9, step=0.1
        ),
        'verbosity': -1 # oh my god shut up
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model = LGBMClassifier(objective="binary", **param_grid)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="binary_error",
            callbacks=[
                LightGBMPruningCallback(trial, "binary_error")
            ],  # Add a pruning callback
        )
        preds = model.predict(X_test)  # Changed this line
        cv_scores[idx] = accuracy_score(y_test, preds)  # Using accuracy on predicted labels


    print(cv_scores)
    return 1 - cv_scores[-1]

study = optuna.create_study(study_name = 'START AGAIN', direction = 'minimize')
study.optimize(objective, n_trials=100)

print('Best hyperparameters: ', study.best_params)
print('Best score: ', study.best_value)

In [ ]:

DS = Dataset(X_train, label = y_train)

def objective(trial):
    params = {
        'objective': 'binary',
        "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 0.95, step=0.1),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 0.95, step=0.1),
        'eval_metric': 'error',
        'random_state': 42
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = LGBMClassifier(**params)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="binary_logloss",
            early_stopping_rounds=100,
            callbacks=[
                LightGBMPruningCallback(trial, "binary_logloss")
            ],  # Add a pruning callback
        )
        preds = model.predict_proba(X_test)
        cv_scores[idx] = accuracy_score(y_test, preds)

    return cv_scores[-1]

study = optuna.create_study(study_name = 'START AGAIN', direction = 'maximize')
study.optimize(objective, n_trials=100)

print('Best hyperparameters: ', study.best_params)
print('Best score: ', study.best_value)